### **IMPORTS**

In [1]:
import numpy as np
import re
# import igraph as ig

### Check Regex Validity

In [56]:
def is_regex_valid(regex):

	# 1- Check that the characters in regex are within the valid set of characters
	# 2- check that all brackets are closed
	regex_operations = ['|', '(', ')', '[', ']', '.', '?', '*', '+', '-', '\\\\']
	bracket,parenthesis = 0, 0

	for char in regex:
		if not char.isalnum() and char != ' ' and char not in regex_operations:
			return False
		
		if char == '(':
			bracket += 1
		elif char == ')':
			bracket -= 1
		elif char == '[':
			parenthesis += 1
		elif char == ']':
			parenthesis -= 1

	if bracket != 0 or parenthesis != 0:
		return False
	
	return True

def is_regex_valid2(regex):

	# 1- Check that the characters in regex are within the valid set of characters
	# 2- check that all brackets are closed
	regex_operations = ['|', '(', ')', '[', ']', '.', '?', '*', '+', '-', '\\\\']
	bracket,parenthesis = 0, 0

	for i,char in enumerate(regex):
		if not char.isalnum() and char != ' ' and char not in regex_operations:
			return False
		
		if char == '(':
			bracket += 1
		elif char == ')':
			bracket -= 1
		elif char == '[':
			parenthesis += 1
		elif char == ']':
			parenthesis -= 1

		# check for valid ranges -> Assume we use simple range [a-z]
		if char == '[':
			# print(ord(regex[i+1]), ord(regex[i+3]))
			# check if char/num then - then char/num
			if  i < len(regex) and not (regex[i+1].isalnum() and regex[i+2] == '-' and regex[i+3].isalnum()):
				# print(regex[i+1], regex[i+3])
				return False
			# check if char 1 is before char 2
			if ord(regex[i+1]) > ord(regex[i+3]):
				print('aa')
				return False
		# - has to be inside a []
		if char == '-':
			if len(regex) < 5 or i-2 >= len(regex) and regex[i-2] !='[':
				return False

	if bracket != 0 or parenthesis != 0:
		return False
	
	return True
	
validity_check = is_regex_valid2('a-z')
print(ord('a'))
print(validity_check)


97
False


### Parse Regex to parts

In [11]:
# We will parse the regex to a list of operations to represent it in FSM
# def parse_regex(regex):
#     regexList = []
#     for ch in regex:
#         if 
#     return regexList

class POSTFIX:
    def __init__(self, regex):
        self.regex = regex
        self.postfix = self.shunt_yard(regex)
    # def get_postfix(self):
    #     return self.postfix
    def shunt_yard(self, regex):
        # Map each operator to its precedence level.
        # The operators supported by this code are * (kleene star), + (one or more), ? (zero or one), . (concatenation), and | (alternation).
        operators = {'*': 5, '+': 4, '?': 5, '.': 2, '|': 1}
        # postfix will eventually store the postfix notation of the input regular expression,.
        # stack is used as an intermediate stack in the Shunting Yard algorithm.
        postfix, stack = "", ""
        # Check if the regular expression contains any character classes (denoted by square brackets).
        # If a character class is found, the function converts it to an alternation between the characters inside the class.
        # For example, the character class [abc] would be converted to the regular expression (a|b|c).
        # This is done using a while loop that iterates over the characters of the regular expression, finds the opening and closing brackets of the character class, and replaces the contents of the class with an alternation.
        for i in range(len(regex)):
            c = regex[i]
            if c == '[':
                j = i + 1
                while regex[j] != ']':
                    if regex[j].isalnum() and regex[j + 1].isalnum():
                        regex = regex[:j + 1] + '|' + regex[j + 1:]
                    j += 1

        # Replace all remaining square brackets with parentheses.
        # This is done because parentheses are used to group sub-expressions in regular expressions
        regex = regex.replace('[', '(')
        regex = regex.replace(']', ')')

        print("postfix1: ", regex)

        # Replace any hyphen character (-) in the regular expression with an alternation between the characters on either side of the hyphen.
        # For example, the expression a-z would be converted to (a|b|c|...|y|z).
        # This is done using another for loop that iterates over the characters of the regular expression, finds the hyphen character, and replaces it with an alternation.
        hyphen_count = regex.count('-')
        for i in range(hyphen_count):
            for j in range(len(regex)):
                c = regex[j]
                if c == '-':
                    final = regex[j + 1]
                    first = regex[j - 1]
                    temp_list = ''
                    for k in range(int(ord(final) - ord(first))):
                        temp_list = temp_list + '|'
                        char = chr(ord(first) + k + 1)
                        temp_list = temp_list + char
                    regex = regex[0: j] + temp_list + regex[j + 2:]
                    break
        print("postfix2: ", regex)
        # Insert a concatenation operator (.) between any two adjacent characters(characters that are not operators), unless the characters are already separated by an operator, or the second character is an opening parenthesis.
        dotIndices = []
        for i in range(len(regex) - 1):
            startOps = [')', "*","+", "*"]
            endOps = ["*", "+", ".", "|", ")"]
            if regex[i] in startOps and regex[i+1] not in endOps:
                dotIndices.append(i)
            elif regex[i].isalnum() and (regex[i+1].isalnum() or regex[i+1] == '('):
                dotIndices.append(i)
        
        for i in range(len(dotIndices)):
            regex = regex[:dotIndices[i] + 1 + i] + '.' + regex[dotIndices[i] + 1 + i:]
        print("postfix3: ", regex)
        # Iterate over each character of the regular expression and performs the Shunting Yard algorithm.
        for i in range(len(regex)):
            c = regex[i]
            # If the character is an opening parenthesis, push it onto the stack.
            if c == '(':
                stack = stack + c
            # If the character is a closing parenthesis, pop operators off the stack and append them to the postfix string until an opening parenthesis is found. Then pop the opening parenthesis from the stack.
            elif c == ')':
                while stack[-1] != '(':
                    # places the character at the end of the stack in the postfix expression
                    postfix = postfix + stack[-1]
                    # [:-1] denotes up to or including the last character
                    stack = stack[:-1]
                stack = stack[:-1]  # removes the open bracket in the stack

            # If the character is an operator, pop operators off the stack and append them to the postfix string as long as they have higher or equal precedence to the current operator. Then push the current operator onto the stack.
            elif c in operators:
                while stack and operators.get(c, 0) <= operators.get(stack[-1], 0):
                    postfix, stack = postfix + stack[-1], stack[:-1]
                stack = stack + c

            # If the character is a operand (i.e. not an operator or parenthesis), append it to the postfix string.
            else:
                postfix = postfix + c
        # After iterating over all characters of the regular expression, the function pops any remaining operators off the stack and appends them to the postfix string.
        while stack:
            postfix, stack = postfix + stack[-1], stack[:-1]
        print("postfix5: ", regex)

        # Finally, the function returns the postfix notation of the input regular expression.
        return postfix

In [12]:
#create a postfix object
post = POSTFIX("(b?a)")
print(post.postfix)

postfix1:  (b?a)
postfix2:  (b?a)
postfix3:  (b?a)
postfix5:  (b?a)
ba?


In [5]:
def infix_to_postfix(infix_regex):
    precedence = {'*': 3, '.': 2, '|': 1, '(': 0}
    output = []
    operator_stack = []

    for char in infix_regex:
        if char.isalnum():
            output.append(char)
        elif char == '(':
            operator_stack.append(char)
        elif char == ')':
            while operator_stack[-1] != '(':
                output.append(operator_stack.pop())
            operator_stack.pop()  # Discard the '('
        else:
            while operator_stack and precedence[operator_stack[-1]] >= precedence[char]:
                output.append(operator_stack.pop())
            operator_stack.append(char)

    while operator_stack:
        output.append(operator_stack.pop())

    return ''.join(output)

# Example usage:
infix_regex = input("Enter an infix regular expression: ")
postfix_regex = infix_to_postfix(infix_regex)
print("Postfix regular expression:", postfix_regex)


Postfix regular expression: 


In [6]:
# Function to convert the infix expression to postfix
def isOperator(c):
    return (not c.isalpha()) and (not c.isdigit())
# Function to get the priority of operators
def getPriority(c):
    if c == '-' or c == '+':
        return 1
    elif c == '*' or c == '/':
        return 2
    elif c == '^':
        return 3
    return 0
 
def infix_to_postfix(infix):
    infix = '(' + infix + ')'
    l = len(infix)
    char_stack = []
    output = ""
 
    for i in range(l):
         
        # Check if the character is alphabet or digit
        if infix[i].isalpha() or infix[i].isdigit():
            output += infix[i]
             
        # If the character is '(' push it in the stack
        elif infix[i] == '(':
            char_stack.append(infix[i])
         
        # If the character is ')' pop from the stack
        elif infix[i] == ')':
            while char_stack[-1] != '(':
                output += char_stack.pop()
            char_stack.pop()
         
        # Found an operator
        else:
            if isOperator(char_stack[-1]):
                if infix[i] == '^':
                    while getPriority(infix[i]) <= getPriority(char_stack[-1]):
                        output += char_stack.pop()
                else:
                    while getPriority(infix[i]) < getPriority(char_stack[-1]):
                        output += char_stack.pop()
                char_stack.append(infix[i])
 
    while len(char_stack) != 0:
        output += char_stack.pop()
    return output
 

In [7]:
regex = "a(b|c)*"
postfix = infix_to_postfix(regex)
print(postfix)  # Output: ab|c*


abc|*


### Make a Class for NFA

In [8]:
# Each state has a list of transitions and epsilon transitions
# We have 2 types of states accepting and non accepting
class State:
	id = 0
	def __init__(self):
		self.id = State.id
		State.id += 1
		self.transitions = []
		self.accepting = False
		self.start = False

		self.epsilon_transitions = [] # NO7OTAHAA TRANSITION 3ADY?
	
	# Print states override
	def __str__(self):
		output = "State: " + str(id(self)) + "\n"
		output += "  Transitions: " + str(self.transitions) + "\n"
		output += "  Epsilon transitions: " + str(self.epsilon_transitions) + "\n"
		output += ("  Accepting? " + str(self.accepting) + "\n")
		output += ("  Saart? " + str(self.accepting) + "\n")
		return output

In [9]:
# NFA class
	# Consists of
		# 1. States ( eaach has its transitions and epsilon transitions)
		# 2. Start State
		# 3. Final State			 

	# Operations that can be done on them
		# 1. Concatenation
		# 2. Union
		# 3. Kleene Star
		# 4. Positive Closure

class NFA:
	def __init__(self, start_state = None, final_state = None, states = [] ) -> None:
		self.states = states
		self.start_state = start_state
		self.final_state = final_state

	# msh 3rfa hn7tag dool walla la2
		####################
	def add_state(self, state):
		self.states.append(state)

	def add_transition(self, from_state, to_state, transition): #transition can be epsilon
		from_state.transitions.append((to_state, transition))

	def add_epsilon_transition(self, from_state, to_state):
		from_state.epsilon_transitions.append(to_state)
    	####################
	 
	def base_nfa(self, symbol): #NFA for a single symbol
		if symbol == "EPSILON":
			start_state = State()
			accept_state = State()
			start_state.epsilon_transitions.append(accept_state)
			return NFA(start_state=start_state, final_state=accept_state)
		else:
			start_state = State()
			accept_state = State()
			accept_state.accepting = True
			start_state.transitions.append((accept_state, symbol))
			return NFA(start_state=start_state, final_state=accept_state)

	def concatenate(self, nfa1, nfa2): # NFA for (A.B)
		nfa1.final_state.accepting = False  
		nfa1.final_state.epsilon_transitions.append(nfa2.start_state)
		self.start_state = nfa1.start_state
		self.final_state = nfa2.final_state
		# self.states.extend(set(nfa1.states + nfa2.states))  # Remove duplicates
		return self
	
	def union(self, nfa1, nfa2): # NFA for (A|B)
		# Add new start and accept states
		start_state = State()
		accept_state = State()
		accept_state.accepting = True
		# Add epsilon transitions from new start state to the start states of nfa1 and nfa2
		start_state.epsilon_transitions.append(nfa1.start_state)
		start_state.epsilon_transitions.append(nfa2.start_state)
		# Add epsilon transitions from the final states of nfa1 and nfa2 to the new accept state
		nfa1.final_state.epsilon_transitions.append(accept_state)
		nfa2.final_state.epsilon_transitions.append(accept_state)
		# Set the new start and accept states of the result NFA
		self.start_state = start_state
		self.final_state = accept_state
		# self.states.extend(set(nfa1.states + nfa2.states + [start_state, accept_state])) # Remove duplicates
		return self
	
	def kleene_star(self, nfa): # NFA for (A*)
		# Add new start and accept states
		start_state = State()
		accept_state = State()
		accept_state.accepting = True
		# Add epsilon transitions from new start state to the start state of nfa
		start_state.epsilon_transitions.append(nfa.start_state)
		# Add epsilon transitions from the final state of nfa to the new accept state (for more than one repetition)
		nfa.final_state.epsilon_transitions.append(nfa.start_state)
		# Add epsilon transitions from the final state of nfa to the new accept state (for zero repetition)
		nfa.final_state.epsilon_transitions.append(accept_state)
		self.start_state = start_state
		self.final_state = accept_state
		# self.states.extend(set(nfa.states + [start_state, accept_state])) # Remove duplicates
		return self

	########## NB Positive closure can be done using a kleen star and concatenation

	def thompson(self, regex):
		"""
		Converts a regular expression in postfix notation to an NFA.
		"""
		nfa_stack = []
		for symbol in regex:
			if symbol.isalpha():  # Single character
				print(self.base_nfa(symbol))
				return
				nfa_stack.append(self.base_nfa(symbol))
			elif symbol == '.':  # Concatenation
				nfa2 = nfa_stack.pop()
				nfa1 = nfa_stack.pop()
				nfa_stack.append(self.concatenate(nfa1, nfa2))
			elif symbol == '|':  # Union
				nfa2 = nfa_stack.pop()
				nfa1 = nfa_stack.pop()
				nfa_stack.append(self.union(nfa1, nfa2))
			elif symbol == '*':  # Kleene star
				nfa = nfa_stack.pop()
				nfa_stack.append(self.kleene_star(nfa))
			elif symbol == '+':  # Positive closure (A+)
				nfa = nfa_stack.pop()  # Get the NFA for A
				nfa_stack.append(self.concatenate(nfa, self.kleene_star(nfa)))  # A followed by A*
		return nfa_stack.pop()  # Final NFA
	
	def __str__(self):
		output = "Start state: " + str(self.start_state.id) + "\n" 
		output += "Final state: " + str(self.final_state.id) + "\n" 
		output += "States:\n"
		for state in self.states:
			output += str(state) + "\n"
		return output

nfa = NFA()
nfa.base_nfa("a")
print(nfa)

AttributeError: 'NoneType' object has no attribute 'id'

### Thompson's Rules

In [ ]:
# def thompson_algorithm(regex):
#     stack = []
#     for char in regex:
# 		if char.isalnum() or char == ' ':
# 			stack.append(NFA.base_nfa(char))
# 		elif char == '.':
# 			nfa2 = stack.pop()
# 			nfa1 = stack.pop()
# 			stack.append(NFA.concatenate(nfa1, nfa2))
# 		elif char == '|':
# 			nfa2 = stack.pop()
# 			nfa1 = stack.pop()
# 			stack.append(NFA.union(nfa1, nfa2))
# 		elif char == '*':
# 			nfa = stack.pop()
# 			stack.append(NFA.kleene_star(nfa))
#     pass

### Write Result to JSON

In [ ]:
def write_FSM(fsm, filename = "fsm.json"):
    pass

## MAIN

In [ ]:
# 1. Get user Input
regex = input("Enter a regex: ")

# 2. Check if the regex is valid
if not is_regex_valid(regex):
	print("Invalid regex")

nfa = NFA()
print(nfa.thompson(regex))
# # 3. Parse the regex
# regex = parse_regex(regex)

# # 4. Implement Thompson's Algorithm
# fsm = thompson_algorithm(regex)

# # 5. Write the FSM to a file
# write_FSM(fsm)


Start state: 4
Final state: State: 2079234431184
  Transitions: []
  Epsilon transitions: []
  Accepting: True

States:

None
